# Imports

In [1]:
import pyaurorax
import datetime
import pprint
import pandas as pd

aurorax = pyaurorax.PyAuroraX()

# Search for data product records

In [2]:
# set values
start = datetime.datetime(2020, 2, 1, 0, 0, 0)
end = datetime.datetime(2020, 2, 5, 23, 59, 59)
programs = ["trex"]
instrument_types = ["RGB ASI"]

In [3]:
# perform search
s = aurorax.search.data_products.search(start, end, programs=programs, verbose=True)

[2024-11-27 21:41:08.636725] Search object created
[2024-11-27 21:41:08.691670] Request submitted
[2024-11-27 21:41:08.691757] Request ID: b056732e-e0fa-4fc2-9973-8b0d36646557
[2024-11-27 21:41:08.691784] Request details available at: https://api.aurorax.space/api/v1/data_products/requests/b056732e-e0fa-4fc2-9973-8b0d36646557
[2024-11-27 21:41:08.691809] Waiting for data ...
[2024-11-27 21:41:10.125926] Checking for data ...
[2024-11-27 21:41:10.549506] Data is now available
[2024-11-27 21:41:10.549751] Retrieving data ...
[2024-11-27 21:41:10.999211] Retrieved 3.4 MB of data containing 1400 records


In [4]:
# output data as a pandas dataframe
data_products = [d.__dict__ for d in s.data]
df = pd.DataFrame(data_products)
df.sort_values("start")

,data_source,data_product_type,start,end,url,metadata
0,"DataSource(identifier=104, program='trex', pla...",montage,2020-02-01T00:00:00,2020-02-01T00:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'montage_type': 'hourly', 'imaging_end_time':..."
26,"DataSource(identifier=103, program='trex', pla...",montage,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'montage_type': 'daily', 'imaging_end_time': ..."
27,"DataSource(identifier=101, program='trex', pla...",montage,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'montage_type': 'daily', 'imaging_end_time': ..."
28,"DataSource(identifier=101, program='trex', pla...",keogram,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'daily_moviederived', 'imagin..."
29,"DataSource(identifier=102, program='trex', pla...",movie,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'movie_type': 'real-time daily', 'imaging_end..."
...,...,...,...,...,...,...
1385,"DataSource(identifier=96, program='trex', plat...",montage,2020-02-05T13:00:00,2020-02-05T13:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'montage_type': 'hourly', 'imaging_end_time':..."
1384,"DataSource(identifier=104, program='trex', pla...",keogram,2020-02-05T13:00:00,2020-02-05T13:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'hourly', 'imaging_end_time':..."
1390,"DataSource(identifier=95, program='trex', plat...",keogram,2020-02-05T13:00:00,2020-02-05T13:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'wavelength': 486.1, 'keogram_type': 'hourly'..."
1398,"DataSource(identifier=96, program='trex', plat...",keogram,2020-02-05T14:00:00,2020-02-05T14:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'hourly', 'imaging_end_time':..."


# Do the search step-by-step

Under the hood, the AuroraX API performs a data product search asynchronously. Note that this does not mean that it can be done using a Python async method; it means that PyAuroraX does more than just a single HTTP request against the AuroraX API when doing a search. With the API operating this way, it adds some more complexity within PyAuroraX but also opens the search up to some very important capabilities. The main capability enabled by this architecture is the ablity to perform queries for a large timeframe, and/or for a large number of data sources. Queries like this can easily take several minutes, and can return hundreds of megabytes - or eve gigabytes - of data. A conventional HTTP request would normally timeout because of this, ultimately failing to complete the search.

Instead of using the `pyaurorax.data_products.search` method which wraps all logic into an easy function, you can also perform a data product search step-by-step if you want more control over the process. Below, we do a search in this manner.

In [5]:
# set up the search parameters
start = datetime.datetime(2020, 2, 1, 0, 0, 0)
end = datetime.datetime(2020, 2, 5, 23, 59, 59)
programs = ["trex"]
instrument_types = ["RGB ASI"]

In [6]:
# create the Search object
s = pyaurorax.search.DataProductSearch(aurorax, start, end, programs=programs, instrument_types=instrument_types)
print(s)

DataProductsSearch(executed=False, completed=False, request_id='')


In [7]:
# execute the search
s.execute()
print(s)

DataProductsSearch(executed=True, completed=False, request_id='63e09ea3-2465-4ea8-affe-bb17170f815d')


In [8]:
# get request status
s.update_status()
pprint.pprint(s.status)

{'logs': [{'level': 'debug',
           'summary': 'Search request arrived and saved',
           'timestamp': '2024-11-27T21:41:14.35348'},
          {'level': 'info',
           'summary': 'Starting query for 63e09ea3-2465-4ea8-affe-bb17170f815d',
           'timestamp': '2024-11-27T21:41:14.356161'},
          {'level': 'info',
           'summary': 'Finished query in 0.19 seconds.',
           'timestamp': '2024-11-27T21:41:14.542963'},
          {'level': 'debug',
           'summary': 'Starting to write data to file',
           'timestamp': '2024-11-27T21:41:14.545608'},
          {'level': 'info',
           'summary': 'Finished writing data, found 820 records in 0.11 '
                      'seconds.',
           'timestamp': '2024-11-27T21:41:14.655424'},
          {'level': 'debug',
           'summary': 'File size is 2107015 bytes.',
           'timestamp': '2024-11-27T21:41:14.658928'},
          {'level': 'info',
           'summary': 'Finished search in 0.31 seconds and 

In [9]:
# view just the logs for the request (update the status beforehand, but we do that in the above cell)
pprint.pprint(s.logs)

[{'level': 'debug',
  'summary': 'Search request arrived and saved',
  'timestamp': '2024-11-27T21:41:14.35348'},
 {'level': 'info',
  'summary': 'Starting query for 63e09ea3-2465-4ea8-affe-bb17170f815d',
  'timestamp': '2024-11-27T21:41:14.356161'},
 {'level': 'info',
  'summary': 'Finished query in 0.19 seconds.',
  'timestamp': '2024-11-27T21:41:14.542963'},
 {'level': 'debug',
  'summary': 'Starting to write data to file',
  'timestamp': '2024-11-27T21:41:14.545608'},
 {'level': 'info',
  'summary': 'Finished writing data, found 820 records in 0.11 seconds.',
  'timestamp': '2024-11-27T21:41:14.655424'},
 {'level': 'debug',
  'summary': 'File size is 2107015 bytes.',
  'timestamp': '2024-11-27T21:41:14.658928'},
 {'level': 'info',
  'summary': 'Finished search in 0.31 seconds and found 820 records.',
  'timestamp': '2024-11-27T21:41:14.663198'}]


In [10]:
# wait for the data
s.wait()
s.update_status()

In [11]:
# get data
s.get_data()
print(s.data[0])

# show data as pandas dataframe
data_products = [d.__dict__ for d in s.data]
df = pd.DataFrame(data_products)
df.sort_values("start")

DataProductData(start='2020-02-01T00:00:00', end='2020-02-01T23:59:00', data_product_type='keogram', url='https://data.phys.uc...', metadata={'keogram_type': 'da...}, data_source=DataSource(...))


,data_source,data_product_type,start,end,url,metadata
0,"DataSource(identifier=102, program='trex', pla...",keogram,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'daily_moviederived', 'imagin..."
20,"DataSource(identifier=101, program='trex', pla...",keogram,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'daily_hires', 'imaging_end_t..."
21,"DataSource(identifier=101, program='trex', pla...",keogram,2020-02-01T00:00:00,2020-02-01T23:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'daily_moviederived', 'imagin..."
22,"DataSource(identifier=103, program='trex', pla...",montage,2020-02-01T00:00:00,2020-02-01T00:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'montage_type': 'hourly', 'imaging_end_time':..."
23,"DataSource(identifier=103, program='trex', pla...",keogram,2020-02-01T00:00:00,2020-02-01T00:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'hourly', 'imaging_end_time':..."
...,...,...,...,...,...,...
812,"DataSource(identifier=104, program='trex', pla...",keogram,2020-02-05T13:00:00,2020-02-05T13:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'hourly', 'imaging_end_time':..."
813,"DataSource(identifier=104, program='trex', pla...",montage,2020-02-05T13:00:00,2020-02-05T13:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'montage_type': 'hourly', 'imaging_end_time':..."
817,"DataSource(identifier=101, program='trex', pla...",keogram,2020-02-05T13:00:00,2020-02-05T13:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'hourly', 'imaging_end_time':..."
818,"DataSource(identifier=96, program='trex', plat...",keogram,2020-02-05T14:00:00,2020-02-05T14:59:00,https://data.phys.ucalgary.ca/sort_by_project/...,"{'keogram_type': 'hourly', 'imaging_end_time':..."
